<h1 style='font-size:40px'> Loading and Preprocessing Data with TensorFlow</h1>

<h2 style='font-size:30px'> The Data API</h2>
<div> 
    <ul style='font-size:20px'>
        <li> 
            A Data API é um módulo do Tensor Flow voltado ao tratamento de datasets volumosos. É interessante ser usado quando os dados não cabem na memória RAM ou placa de vídeo.
        </li>
        <li> 
            A classe `Dataset` é onde nossos dados são armazenados.
        </li>
    </ul>
</div>

In [9]:
import tensorflow as tf
import tensorflow.keras as keras

# Criando um `Dataset` a partir de um `tf.range`. 
X = tf.range(10)

# Cada número de `X` será encapsulado em um tensor.
dataset = tf.data.Dataset.from_tensor_slices(X)

for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [73]:
# O mesmo Dataset poderia ter sido montado da seguinte maneira:
dataset = tf.data.Dataset.range(10)

<h3 style='font-size:30px;font-style:italic'> Chaining Transformations</h3>
<div> 
    <ul style='font-size:20px'>
        <li> 
            As funções da classe `Dataset` nunca fazem transformações in-place; sempre retornam um novo objeto.
        </li>
    </ul>
</div>

In [74]:
#  `repeat` repetirá os dados do dataset, enquanto `batch` vai criar batches com n instâncias.
for item in dataset.repeat(3).batch(7):
    # Para evitar que o último batch com os elementos restantes do dataset seja formado, passe `drop_remainder`=True.
    print(item)
    
# Observe que `batch` não embaralha os dados na hora de sua separação.

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int64)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int64)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int64)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int64)
tf.Tensor([8 9], shape=(2,), dtype=int64)


In [39]:
# `map` aplica uma função por elemento.
# `apply` invoca uma transformação a todo o dataset.
list(dataset.map(lambda x: x**2))

[<tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>,
 <tf.Tensor: shape=(), dtype=int64, numpy=16>,
 <tf.Tensor: shape=(), dtype=int64, numpy=81>,
 <tf.Tensor: shape=(), dtype=int64, numpy=256>,
 <tf.Tensor: shape=(), dtype=int64, numpy=625>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1296>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2401>,
 <tf.Tensor: shape=(), dtype=int64, numpy=4096>,
 <tf.Tensor: shape=(), dtype=int64, numpy=6561>]

In [47]:
# `filter`, é bastante eficaz em remover dados indesejados.
list(dataset.filter(lambda x: x<5))

[<tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2>,
 <tf.Tensor: shape=(), dtype=int64, numpy=3>,
 <tf.Tensor: shape=(), dtype=int64, numpy=4>]

<h3 style='font-size:30px;font-style:italic'> Shuffling the Data</h3>
<div> 
    <ul style='font-size:20px'>
        <li> 
            O método `shuffle` embaralha as instâncias com a seguinte lógica: escolhe as primeiras x instâncias do dataset e as agrupa em um buffer. Daí, as mistura e sorteia um dado, quando solicitado. Após a extração, o buffer fica com um espaço sobrando, que é preenchido com a próxima instância do dataset.
        </li>
        <li> 
            Por conta do algoritmo, é contraindicado usar `buffer_size`'s pequenos. A documentação do TF indica, até mesmo, designarmos um valor maior ou igual ao o dataset. Apenas cuidado com a memória RAM!            
        </li>
    </ul>
</div>

In [76]:
# Embaralhando um dataset artificial (`buffer_size`=10).

# `reshuffle_each_iteration` garante que um novo embaralhamento dos dados ocorrerá, caso usemos `repeat`, por exemplo.
dataset = tf.data.Dataset.range(5)
list(dataset.shuffle(5, seed=42, reshuffle_each_iteration=True).repeat(2))

[<tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=4>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>,
 <tf.Tensor: shape=(), dtype=int64, numpy=3>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2>,
 <tf.Tensor: shape=(), dtype=int64, numpy=3>,
 <tf.Tensor: shape=(), dtype=int64, numpy=4>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2>]

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Interleaving lines from multiple files</h4>
<div> 
    <ul style='font-size:20px'>
        <li> 
            A fim de garantirmos a ordem aleatória dos dados no abastecimento de modelos, talvez seja interessante quebrarmos o set de treino em múltiplos arquivos. Assim, os lemos simultaneamente, intercalando as suas linhas. No final, conseguimos ainda aplicar a função `shuffle` para misturar todo o produto da leitura.
        </li>
    </ul>
</div>

In [120]:
# Diretório onde vão ser armazenados os dados.
from os import mkdir
mkdir('data')

In [373]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
X,y = fetch_california_housing(return_X_y=True,as_frame=True)
data = pd.merge(X,y, left_index=True, right_index=True)

for i in range(0, len(data), 1000): # Forma rápida de gerar os arquivos, mesmo sabendo que vou perder algumas linhas.
    data.iloc[i:i+1000].to_csv(f'data/housing_{int(i/1000)}.csv', index=False)

In [122]:
# O método `list_files` localiza um conjunto de arquivos baseado em certo padrão nas strings.
# Retorna um dataset com os paths de todos os arquivos que estão de acordo com a string oferecida.
filepath_dataset = tf.data.Dataset.list_files('data/housing_*.csv', shuffle=True)
list(filepath_dataset)

[<tf.Tensor: shape=(), dtype=string, numpy=b'data/housing_4.csv'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'data/housing_14.csv'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'data/housing_5.csv'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'data/housing_1.csv'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'data/housing_19.csv'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'data/housing_2.csv'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'data/housing_20.csv'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'data/housing_11.csv'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'data/housing_10.csv'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'data/housing_12.csv'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'data/housing_7.csv'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'data/housing_0.csv'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'data/housing_8.csv'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'data/housing_18.csv'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'data/housi

In [136]:
# A leitura dinâmica dos arquivos pode ser feita com o método `interleave`.

# Esse aplica uma função sobre todo o `Dataset` e intercala as suas linhas.
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers,
    # Por padrão, não há uso de paralelismo. Uma linha de cada dataset é lida por vez.
    # Use `num_parallel_calls` se quiser fazer esse processo por multithreading.
    num_parallel_calls=-1
)

In [142]:
# Mas veja: os valores das features estão todos contidos dentro de uma string bytes.

# É necessário ainda fazermos mais um tratamento, coletando os números individualizados.
list(dataset.take(5))

[<tf.Tensor: shape=(), dtype=string, numpy=b'12000,7.5408,3.0,8.493150684931507,1.0342465753424657,519.0,3.5547945205479454,33.93,-117.57,2.719'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'16000,4.0846,52.0,4.821316614420063,0.9561128526645768,819.0,2.5673981191222572,37.74,-122.47,3.336'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'3000,4.1484,10.0,4.791907514450867,0.8439306358381503,447.0,2.5838150289017343,35.3,-119.03,1.029'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'11000,13.466,26.0,8.874233128834355,1.0582822085889572,983.0,3.0153374233128836,33.75,-117.79,5.00001'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'9000,4.0,49.0,6.866295264623956,1.0362116991643453,1018.0,2.8356545961002784,34.0,-118.34,2.968'>]

<h3 style='font-size:30px;font-style:italic'> Preprocessing the Data</h3>
<div> 
    <ul style='font-size:20px'>
        <li> 
            Criaremos uma função que recebe uma linha do dataset e a retorna devidamente tratada.
        </li>
    </ul>
</div>

In [374]:
# Médias e desvios-padrão das colunas para padronização.
X_mean, X_std = X.mean().to_numpy(), X.std().to_numpy()

# Função de tratamento dos dados.
def preprocess(line):
    n_inputs = 8 # Quantidade de colunas presentes.
    # Lista com os valores-padrão a serem imputados nas células caso haja NaN's. Um tensor vazio será gerado caso 
    # o nulo ocorra em uma target-variable, acarretando em um erro.
    default_values = [0.]*n_inputs + [tf.constant([], dtype=tf.float32)]
    # Decodificando a linha. 'default_values' será usado na ocorrência de vazios.
    fields = tf.io.decode_csv(line, record_defaults=default_values) # `fields é uma lista de tensores que contêm um único número.
    # A função `stack` empilha os valores dos tensores em um único tensor unidimensional.
    x, y = tf.stack(fields[:-1]), tf.stack(fields[-1:])
    # Retornando as variáveis independentes padronizadas juntamente com as dependentes.
    return (x-X_mean)/X_std, y

In [375]:
preprocess(b'2.23,1.73,10.98,4.56,9.01,0.01,0.03,20.31,5.41')

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([ -0.8635921 ,  -2.1381242 ,   2.2435777 ,   7.3079667 ,
         -1.250785  ,  -0.29468906, -16.66791   ,  69.81657   ],
       dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([5.41], dtype=float32)>)

<h3 style='font-size:30px;font-style:italic'> Putting Everything Together</h3>
<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Prefetching</h4>
<div> 
    <ul style='font-size:20px'>
        <li> 
            O método `prefetch` contribui para a performance de nosso programa. Enquanto um dado é processado, o método faz com que a próxima instância (ou batch) passe pelo tratamento em paralelo, economizando tempo.
        </li>
    </ul>
</div>

<center>
    <img src='prefetch.png'>
</center>

In [376]:
range_dataset = tf.data.Dataset.range(10)

# Preparando a próxima instância proveniente de 'range_dataset'.
range_dataset.prefetch(1)

# Preparando o próximo batch criado a partir de 'range_dataset'.
range_dataset.repeat(3).prefetch(1)

<PrefetchDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [377]:
# Finalizando a seção com uma função  que realiza todos os passos demonstrados.
def csv_reader_dataset(filepath:str, cycle_length:int=5, buffer_size:int=1000, repeat:int=3, batch_size:int=32):
    filepaths = tf.data.Dataset.list_files(filepath) # Coletando todos os arquivos correspondentes ao padrão `filepath`.
    # Lendo `cycle_length` arquivos, pulando as suas primeiras linhas. 
    dataset = filepaths.interleave(lambda x: tf.data.TextLineDataset(x).skip(1), 
                                   cycle_length=cycle_length, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).cache() # Tratando os dados e os armazenando em cache.
    dataset = dataset.shuffle(buffer_size).repeat(repeat) # Embaralhando e repetindo os dados.
    # A função segregará os dados em batches. À cada batch sendo processado, o próximo é criado.
    return dataset.batch(batch_size, num_parallel_calls=tf.data.AUTOTUNE).prefetch(1)

<div> 
    <ul style='font-size:20px'>
        <li> 
            Usamos `cache` após o tratamento, mas antes do shuffling e batching. Isso garantirá que a mistura e batches serão distintos a cada iteração. 
        </li>
    </ul>
</div>

In [378]:
# Teste da função.
csv_reader_dataset('data/housing_*.csv')

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 8), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

<h3 style='font-size:30px;font-style:italic'> Using the Dataset with tf.keras</h3>
<div> 
    <ul style='font-size:20px'>
        <li> 
            Com a Pipeline montada, é hora de criar uma rede neural com os dados.
        </li>
    </ul>
</div>

In [379]:
# Caminhos para os arquivos de treino, validação e teste.
train_paths = [f'data/housing_{i}.csv' for i in range(12)]
valid_paths = [f'data/housing_{i}.csv' for i in range(12, 16)]
test_paths = [f'data/housing_{i}.csv' for i in range(16, 21)]

In [380]:
# Sets com os dados.
train_set = csv_reader_dataset(train_paths)
valid_set = csv_reader_dataset(valid_paths)
test_set = csv_reader_dataset(test_paths)

In [382]:
# Hora demontar o modelo.
def data_api_model():
    model = keras.models.Sequential()
    # Camada de Input.
    model.add(keras.layers.Input(shape=[8]))
    # Hidden Layers.
    for _ in range(5): 
        model.add(keras.layers.Dense(40, activation='elu', kernel_initializer='lecun_normal'))
    # Output Layer
    model.add(keras.layers.Dense(1))
    return model

In [385]:
model = data_api_model()
model.compile(optimizer='nadam', loss='mse')
model.fit(train_set, batch_size=20, epochs=10, validation_data=valid_set )

Epoch 1/10
1125/1125 [==============================] - 10s 6ms/step - loss: 0.4308 - val_loss: 0.7267
Epoch 2/10
1125/1125 [==============================] - 7s 6ms/step - loss: 0.3109 - val_loss: 0.6991
Epoch 3/10
1125/1125 [==============================] - 7s 6ms/step - loss: 0.2969 - val_loss: 0.6759
Epoch 4/10
1125/1125 [==============================] - 7s 6ms/step - loss: 0.2858 - val_loss: 0.8068
Epoch 5/10
1125/1125 [==============================] - 7s 6ms/step - loss: 0.2782 - val_loss: 0.5866
Epoch 6/10
1125/1125 [==============================] - 7s 6ms/step - loss: 0.2722 - val_loss: 0.6028
Epoch 7/10
1125/1125 [==============================] - 7s 6ms/step - loss: 0.2639 - val_loss: 0.5513
Epoch 8/10
1125/1125 [==============================] - 7s 6ms/step - loss: 0.2608 - val_loss: 0.5665
Epoch 9/10
1125/1125 [==============================] - 7s 6ms/step - loss: 0.2563 - val_loss: 0.5633
Epoch 10/10
1125/1125 [==============================] - 7s 6ms/step - loss: 0.25

<h3 style='font-size:30px;font-style:italic'> The TFRecord Format</h3>
<div> 
    <ul style='font-size:20px'>
        <li> 
            O formato de arquivo TFRecord é conhecido pela sua flexibilidade. É capaz de armazenar diferentes tipos de dados, como textos e áudios.
        </li>
    </ul>
</div>

In [386]:
# Criando um pequeno arquivo TFRecord
with tf.io.TFRecordWriter('my_tfrecord.tfrecord') as f:
    f.write('First Row')
    f.write('Second Row')
    f.close()

In [391]:
# Agora, lendo o arquivo.
file = tf.data.TFRecordDataset('my_tfrecord.tfrecord')

for item in file:
    print(item)

tf.Tensor(b'First Row', shape=(), dtype=string)
tf.Tensor(b'Second Row', shape=(), dtype=string)


<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Compressed TFRecord Files</h4>
<div> 
    <ul style='font-size:20px'>
        <li> 
            Conseguimos usar métodos de compressão nos arquivos .tfrecord.
        </li>
    </ul>
</div>

In [392]:
options = tf.io.TFRecordOptions(compression_type='GZIP')
with tf.io.TFRecordWriter('my_tfrecord2.tfrecord', options=options) as f:
    f.write('First Row')
    f.write('Second Row')
    f.close()

In [393]:
# Ao carregar esse mesmo arquivo, você deve informar o método que o comprimiu.
tf.data.TFRecordDataset('my_tfrecord2.tfrecord', compression_type='GZIP')

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

<h3 style='font-size:30px;font-style:italic'> A Brief Introduction to Protocol Buffers</h3>
<div> 
    <ul style='font-size:20px'>
        <li> 
            O protocol buffer (protobuf) é uma espécie de formatação de arquivos desenvolvida pelo Google. Sua sintaxe é similar com a do JSON, mas é muito mais rápida de ser lida, pois seus registros são armazenados em bytes.
        </li>
        <li> 
            Nos arquivos TFRecord, os dados pertencentes às instâncias são armazenados no formato de protobufs,
        </li>
    </ul>
</div>

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> TensorFlow Protobufs</h4>
<div> 
    <ul style='font-size:20px'>
        <li> 
            Somos capazes de construir os nossos próprios protobufs com o módulo `tf.train`. 
        </li>
    </ul>
</div>

In [407]:
# A classe `Example` representa uma única instância do dataset. Essa contém o argumento 'features', que admite uma classe `Features`.
# Essa, por sua vez, tem o campo 'feature', que recebe um dicionário <nome_feature>:Feature(ClasseDataType(valor_feature)).
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Example, Feature, Features

# Montando um protobuf-exemplo.
person_example = Example(
    features=Features(
        feature={
            'id':Feature(int64_list=Int64List(value=[1])),
            'name':Feature(bytes_list=BytesList(value=[b'Alice'])),
            'emails':Feature(bytes_list=BytesList(value=[b'email@gmail.com']))
    }))
person_example

features {
  feature {
    key: "emails"
    value {
      bytes_list {
        value: "email@gmail.com"
      }
    }
  }
  feature {
    key: "id"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "name"
    value {
      bytes_list {
        value: "Alice"
      }
    }
  }
}

In [400]:
# Vamos registrar a instância em um arquivo .tfrecord.
with tf.io.TFRecordWriter('my_contacts_record.tfrecord') as f:    
    f.write(person_example.SerializePartialToString())

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Loading and Parsing Examples</h4>
<div> 
    <ul style='font-size:20px'>
        <li> 
            Para carregar o protobuf de um exemplo, use a função `parse_single_example` dentro de um loop envolvendo a classe `TFRecordDataset`.
        </li>
    </ul>
</div>

In [402]:
# O método requer um dicionário que mapeia o nome da feature para uma classe que sinaliza informações como shape, datatype e default_value.

# 'id' e 'name' sempre serão listas de um único elemento. Por isso, a classe `FixedLenFeature` é usada.
# Por outro lado, a lista de 'emails' admite qualquer quantidade de strings. Dessa forma, recorremos à VarLenFeature, que exige apenas
# que o data type seja informado.
description = {
        'id':tf.io.FixedLenFeature([], tf.int64, default_value=0),
        'name':tf.io.FixedLenFeature([], tf.string, default_value=''),
        'emails':tf.io.VarLenFeature(tf.string)
}

# Lendo o `Example` do arquivo há pouco gerado.
for serialized_example in tf.data.TFRecordDataset('my_contacts_record.tfrecord'):
    parsed_example = tf.io.parse_single_example(serialized_example, description)

In [404]:
# `parsed_example` é um dicionário <feature>:<Tensor_feature>.
parsed_example

{'emails': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f9bbc10b6a0>,
 'id': <tf.Tensor: shape=(), dtype=int64, numpy=1>,
 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'Alice'>}

In [405]:
# Observe que a feature informada como de tamanho variável é retornada como um tensor esparso. 

# Podemos ler os seus valores tanto com `tf.sparse.to_dense`, quanto usando o atributo 'values' do tensor.
parsed_example['emails'].values

<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'email@gmail.com'], dtype=object)>

<div> 
    <ul style='font-size:20px'>
        <li> 
            Mas é claro que não leríamos uma única instância do dataset em projetos reais. Use  `parse_example` para ler múltiplas linhas.
        </li>
    </ul>
</div>

In [481]:
# Para exemplificar o uso, vamos recorrer ao `fetch_california_housing`, já que esse possui bastantes linhas.
data = pd.read_csv('data/housing_0.csv')

# Pequena função para a criação de uma classe `Example` para cada linha de 'data'.
def create_example(line, columns=data.columns):
    example = Example(
        features=Features(
            feature={
                column:Feature(float_list=FloatList(value=[line[i]]))
                  for i, column in enumerate(columns)          
    }))
    return example

In [483]:
# Armazenando os dados em um arquivo 'tf.record'.
with tf.io.TFRecordWriter('housing_tf.tfrecord') as f:
    for row in data.values:
        example = create_example(row)
        f.write(example.SerializePartialToString())

In [504]:
# Oferecendo os data types de cada coluna para a leitura do arquivo produzido.
description = {
    col:tf.io.FixedLenFeature([], tf.float32, default_value=0)
    for col in X.columns
}

# Ao invés de lermos instância por instância, usamos `parse_example` para ler batches de 10 dados e embaralhá-los.
for data in tf.data.TFRecordDataset('housing_tf.tfrecord').batch(10).shuffle(10, seed=42):
    parsed_examples = tf.io.parse_example(data, description)

In [506]:
parsed_examples

{'AveBedrms': <tf.Tensor: shape=(10,), dtype=float32, numpy=
 array([1.0095367 , 1.0135417 , 1.0440421 , 0.9763113 , 1.0832196 ,
        1.0850723 , 0.9019608 , 0.97894734, 0.97792023, 1.1014493 ],
       dtype=float32)>,
 'AveOccup': <tf.Tensor: shape=(10,), dtype=float32, numpy=
 array([3.2043598, 2.9739583, 3.297217 , 3.3519459, 2.43588  , 3.165329 ,
        3.0952382, 3.630263 , 3.0762107, 3.3064182], dtype=float32)>,
 'AveRooms': <tf.Tensor: shape=(10,), dtype=float32, numpy=
 array([5.737057 , 5.7625   , 7.635234 , 8.30626  , 5.075716 , 6.9245586,
        5.647059 , 7.075    , 6.5292025, 6.3913045], dtype=float32)>,
 'HouseAge': <tf.Tensor: shape=(10,), dtype=float32, numpy=array([26., 27.,  7.,  9., 10., 23., 26., 13., 22., 26.], dtype=float32)>,
 'Latitude': <tf.Tensor: shape=(10,), dtype=float32, numpy=
 array([37.52, 37.53, 37.53, 37.49, 37.49, 37.48, 37.47, 37.47, 37.55,
        37.55], dtype=float32)>,
 'Longitude': <tf.Tensor: shape=(10,), dtype=float32, numpy=
 array([-12

<h2 style='font-size:30px'> Preprocessing the Input Features</h2>
<div> 
    <ul style='font-size:20px'>
        <li> 
            Vamos aprender aqui a implantar algumas técnicas de preprocessamento de dados pelo TensorFlow.
        </li>
    </ul>
</div>

In [544]:
# O livro ensina a fazer uma Padronização, mas achei que faria mais sentido montar uma camada de Normalização.
class NormLayer(keras.layers.Layer):
    # Chame `adapt` para que as normas do seu dataset sejam mensuradas.
    def adapt(self, input_data):
        # Normas das features são armazenadas em `self.norms`.
        self.norms = tf.linalg.norm(input_data, axis=0, keepdims=True)
        
    def call(self, inputs):
        # Normalizando os dados.
        return inputs / self.norms
    
norm_layer = NormLayer()
norm_layer.adapt(X)
norm_layer.call(X.values)

<tf.Tensor: shape=(20640, 8), dtype=float64, numpy=
array([[ 0.01343958,  0.00912272,  0.00814819, ...,  0.00164245,
         0.00738648, -0.00711444],
       [ 0.01340116,  0.00467261,  0.00727786, ...,  0.00135599,
         0.00738258, -0.00711386],
       [ 0.0117158 ,  0.01157028,  0.00966954, ...,  0.00180101,
         0.00738063, -0.00711502],
       ...,
       [ 0.00274435,  0.00378259,  0.00607316, ...,  0.00149468,
         0.00768873, -0.00705565],
       [ 0.00301427,  0.0040051 ,  0.0062178 , ...,  0.00136458,
         0.00768873, -0.00706148],
       [ 0.00385598,  0.00356009,  0.00613053, ...,  0.00168193,
         0.00767703, -0.00705682]])>

<h3 style='font-size:30px;font-style:italic'> Encoding Categorical Features Using One-Hot Vectors</h3>
<div> 
    <ul style='font-size:20px'>
        <li> 
            Como visto no Capítulo 2, os vetores One-Hot indicam a presença ou não de uma determinada categoria no dataset. O TensorFlow possui mecanismos robustos para lidar com esse tipo de tratament.
        </li>
    </ul>
</div>

In [558]:
# Como ilustração, usaremos as categorias presentes no 'housing.csv'.
df = pd.read_csv('housing.csv')
vocab = df['ocean_proximity'].unique() # Listando as categorias.
indices = tf.range(len(vocab), dtype=tf.int64) # Vetor com o código de cada categoria.
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices) # Inicializador da tabela de Lookup.
num_oov_buckets = 2 # A tabela de Lookup admite até duas novas categorias.
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

<div> 
    <ul style='font-size:20px'>
        <li> 
            A Tabela de Lookup terá duas colunas extras para categorias ausentes no vocabulário informado. Uma vez que um termo desse tipo é encontrado, ele é designado a uma coluna, passando essa a pertencê-la.
        </li>
        <li> 
            Se a quantidade de categorias desconhecidas exceder 'num_oov_buckets', termos distintos serão designados a um mesmo bucket.
        </li>
    </ul>
</div>

In [564]:
# Testando a tabela de Lookup.
categories = np.random.choice(vocab, size=5)
categories = tf.constant(np.append(categories, 'New Cat')) # Acrescentando uma categoria ausente no vocabulário.

# Veja que 'New Cat' assume o índice 6.
table.lookup(categories)

<tf.Tensor: shape=(6,), dtype=int64, numpy=array([4, 0, 4, 4, 4, 6])>

In [566]:
# O output do método `lookup` pode ser aproveitado para fazer o One-Hot Encoding.
# Precismamos informar em `depth` a quantidade de categorias esperadas.
tf.one_hot(table.lookup(categories), depth=len(vocab) + num_oov_buckets)

<tf.Tensor: shape=(6, 7), dtype=float32, numpy=
array([[0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.]], dtype=float32)>

<h3 style='font-size:30px;font-style:italic'> Encoding Categorical Features Using Embeddings</h3>
<div> 
    <ul style='font-size:20px'>
        <li> 
        </li>
    </ul>
</div>

<p style='color:red'> Encoding Categorical Features Using Embeddings
</p>